In [1]:
import requests, json
token = 'secret_kRjcHJDgYxNTPlnl323mI4k6a7tMckr7tPWDPj10LOT'
databaseId = '35f200562f344d02bd942cbb7e51478a'
pageId = ''
update_DB_URL = f'https://api.notion.com/v1/databases/{databaseId}'

headers = {
    "Authorization": "Bearer " + token,
    "Content-Type": "application/json",
    "Notion-Version": "2022-06-28"}

# Helper Functions

In [4]:
def readPage(pageId="88d4370d8f40431dbfa03c938788df80", headers):
    readUrl = f"https://api.notion.com/v1/pages/{pageId}/query"

    res = requests.request("POST", readUrl, headers=headers)
    data = res.json()
    print(res.status_code) 
    
    return data

def readDatabase(databaseId, headers):
    readUrl = f"https://api.notion.com/v1/databases/{databaseId}/query"

    res = requests.request("POST", readUrl, headers=headers)
    data = res.json()
    print(res.status_code)
    # print(res.text)

    with open('./db.json', 'w', encoding='utf8') as f:
        json.dump(data, f, ensure_ascii=False)
    
    return data


def get_idea_from_content_board(db_data, content_board_tag = "Idea"):
    content_list = []
    for i in range(len(db_data["results"])):
        try:
            if db_data["results"][i]["properties"]["Status"]["select"]["name"] == content_board_tag:
                content_list.append(db_data["results"][i]["properties"]["Name"]["title"][0]["plain_text"])
        except:
            continue
    
    return content_list


def get_content_title_idx(content_title):
    length_db = len(db_data["results"])
    for i in range(length_db):
        if db_data["results"][i]["properties"]["Name"]["title"][0]["plain_text"]==content_title:
            content_idx = i
            break
    
    return content_idx


def get_page_in_database(databaseId, headers, page_content_idx):
    db_data = readDatabase(databaseId, headers)
    return db_data["results"][page_content_idx]

def remove_page_metadata(page_data, keys_to_remove):
    for key in keys_to_remove:
        page_data.pop(key)
    
    return page_data

def remove_property_metadata(page_data, property_name, property_type,
                             keys_to_remove):
    """
    Removes unwated property metadata to be able to update
    a page.
    """
    print("Removing property specific meta keys (color and id for select)")
    for key in keys_to_remove:
        page_data["properties"][property_name][property_type].pop(key)
    
    return page_data

def change_page_property_option(page_data,pageId,headers,property_name, property_type, property_dest_name):
    page_data["properties"][property_name][property_type]["name"]=property_dest_name
    return page_data
    
        
def update_page_property(page_data,pageId,headers,property_name, property_type, property_dest_name):
    updateURL = f'https://api.notion.com/v1/pages/{pageId}'
    page_data = change_page_property_option(page_data,pageId,headers,property_name, property_type, property_dest_name)
    updated_page = json.dumps(page_data)
    res = requests.request("PATCH", updateURL, headers=headers, data=updated_page)
    print(res.text)
    print("Content Board updated!")


# def createPage(databaseId, headers):

#     createUrl = 'https://api.notion.com/v1/pages'
    
#     newPageData = {}

#     data = json.dumps(newPageData)
#     # print(str(uploadData))

#     res = requests.request("POST", createUrl, headers=headers, data=data)

#     print(res.status_code)
#     print(res.text)

__Get all the ideas from the Content Board__

In [6]:
db_data = readDatabase(databaseId, headers)
content_status_tags = ["Idea", "Drafts", "Next Improvements", "Edit", "Publish"]
for tag in content_status_tags:
    print(f"Tag: {tag} - ", get_idea_from_content_board(db_data, content_board_tag=tag))

200
Tag: Idea -  ['Automating Notion With Python', 'Automating Content Management with Python and the Notion API', 'Article - 10 Tips for Better ML Dev', 'Pose Tracking']
Tag: Drafts -  []
Tag: Next Improvements -  ['Use this structure for the next code/jupyter notebooks explanation video', 'Audio quality (mic position, volume when editing, etc.)', 'Floating link window inside video', 'Shot/lights improvement', 'Background improvement ']
Tag: Edit -  []
Tag: Publish -  []


In [8]:
page_title = "Automating Notion With Python"
property_name = "Status"
property_type = "select"
property_dest_name = "Drafts"
page_content_idx = get_content_title_idx(page_title)
page_data = db_data["results"][page_content_idx]
page_data["properties"]["Status"]["select"]["name"]
pageId = page_data["id"]
update_PAGE_URL = f'https://api.notion.com/v1/pages/{pageId}'
print(f"Page ID: {pageId}")
print("Removing meta keys (last edit time, create by, etc...)")
page_data = remove_page_metadata(page_data, keys_to_remove=["last_edited_time","created_time","last_edited_by","created_by"])
page_data = remove_property_metadata(page_data, property_name, property_type, keys_to_remove=["id", "color"])
print(f"Updating the {property_name} option...")
update_page_property(page_data,pageId,headers,property_name, property_type, property_dest_name)

Page ID: acb4a4d5-b53c-4622-99ef-6f1663247ff5
Removing meta keys (last edit time, create by, etc...)
Removing property specific meta keys (color and id for select)
Updating the Status option...
{"object":"page","id":"acb4a4d5-b53c-4622-99ef-6f1663247ff5","created_time":"2023-02-02T17:32:00.000Z","last_edited_time":"2023-02-02T17:37:00.000Z","created_by":{"object":"user","id":"bf94c211-4404-4995-bed7-62e120c5e1eb"},"last_edited_by":{"object":"user","id":"4c0ee156-6405-4cc5-af87-66ce26f3835e"},"cover":null,"icon":null,"parent":{"type":"database_id","database_id":"35f20056-2f34-4d02-bd94-2cbb7e51478a"},"archived":false,"properties":{"Date":{"id":"WV~R","type":"date","date":null},"Status":{"id":"%5C%7Bn~","type":"select","select":{"id":"ec45ef06-f1ea-410b-89c3-fd48b9a0a4f1","name":"Drafts","color":"pink"}},"Name":{"id":"title","type":"title","title":[{"type":"text","text":{"content":"Automating Notion With Python","link":null},"annotations":{"bold":false,"italic":false,"strikethrough":fals

In [47]:
get_idea_from_content_board(content_board_tag="Idea")

['Automating Content Management with Python and the Notion API',
 'Pose Tracking']

In [111]:
page_content_idx = get_content_title_idx("Pose Tracking")
page_content_idx

7

In [116]:
update_content_status(databaseId, headers, db_data, destination_status="Drafts", page_title="Pose Tracking", page_content_idx=page_content_idx)

{"object":"error","status":400,"code":"validation_error","message":"body failed validation: body.parent should be defined, instead was `undefined`."}
Content Board updated!
